# Dogs vs. Cats Redux: Kernels Edition

## 2. 모델구성 및 학습

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image, HTML
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import time

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 2.1 Model class
VGG19

In [2]:
class Model_Vgg19:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name

    def build_net(self, image_size):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, image_size, image_size, 3])
            self.Y = tf.placeholder(tf.float32, [None, 1])
            self.learning_rate = tf.placeholder(tf.float32)

            # Convolutional Layer #1
            conv1_1 = tf.layers.conv2d(inputs=self.X, filters=64, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv1_2 = tf.layers.conv2d(inputs=conv1_1, filters=64, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1_2, pool_size=[2, 2], padding="SAME", strides=2)
            
            conv2_1 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2_2, pool_size=[2, 2], padding="SAME", strides=2)
            
            conv3_1 = tf.layers.conv2d(inputs=pool2, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv3_2 = tf.layers.conv2d(inputs=conv3_1, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv3_3 = tf.layers.conv2d(inputs=conv3_2, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv3_4 = tf.layers.conv2d(inputs=conv3_3, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3_3, pool_size=[2, 2], padding="SAME", strides=2)
            
            #conv4_1 = tf.layers.conv2d(inputs=pool3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv4_2 = tf.layers.conv2d(inputs=conv4_1, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv4_3 = tf.layers.conv2d(inputs=conv4_2, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv4_4 = tf.layers.conv2d(inputs=conv4_3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #pool4 = tf.layers.max_pooling2d(inputs=conv4_3, pool_size=[2, 2], padding="SAME", strides=2)
            
            #conv5_1 = tf.layers.conv2d(inputs=pool4, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv5_2 = tf.layers.conv2d(inputs=conv5_1, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv5_3 = tf.layers.conv2d(inputs=conv5_2, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #conv5_4 = tf.layers.conv2d(inputs=conv5_3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            #pool5 = tf.layers.max_pooling2d(inputs=conv5_3, pool_size=[2, 2], padding="SAME", strides=2)
            
            initializer = tf.contrib.layers.xavier_initializer()
            
            # 150 -> 75 -> 38 -> 19 -> 10 -> 5
            # Dense Layer with Relu
            flat6 = tf.reshape(pool3, [-1, 256 * 19 * 19])
            #fc6 = tf.layers.dense(inputs=flat6, units=6400, activation=tf.nn.relu, kernel_initializer=initializer)
            fc6 = tf.layers.dense(inputs=flat6, units=1000, activation=tf.nn.relu, kernel_initializer=initializer)
            dropout6 = tf.layers.dropout(inputs=fc6, rate=0.5, training=self.training)

            flat7 = tf.reshape(dropout6, [-1, 6400])
            #fc7 = tf.layers.dense(inputs=flat7, units=3200, activation=tf.nn.relu, kernel_initializer=initializer)
            fc7 = tf.layers.dense(inputs=flat7, units=500, activation=tf.nn.relu, kernel_initializer=initializer)
            dropout7 = tf.layers.dropout(inputs=fc7, rate=0.5, training=self.training)

            # Logits (no activation) Layer: L7 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout7, units=1)
            
        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

        self.prediction = tf.round(tf.nn.sigmoid(self.logits))
        correct_prediction = tf.equal(self.prediction, self.Y)
        self.correct_count = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, learning_rate, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.learning_rate: learning_rate, self.training: training})

### 2.2 이미지 파일 리스트 가져 오기

In [3]:
DATA_DIR = "../data/"
TRAIN_DIR = DATA_DIR + "train_resize/"
TEST_DIR = DATA_DIR + "test_resize/"

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] 
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]
test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

print("train_dogs: ", len(train_dogs))
print("train_cats: ", len(train_cats))
print("test_images: ", len(test_images))

train_dogs:  12500
train_cats:  12500
test_images:  12500


### 2.3 학습데이터와 valid 데이터 분리

In [4]:
train_dogs = train_dogs[:100]
train_cats = train_cats[:100]

train_dog_cat = train_dogs + train_cats

label_dog = [1 for i in range(len(train_dogs))]
label_cat = [0 for i in range(len(train_cats))]
label = label_dog + label_cat

x_train, x_valid, y_train, y_valid = train_test_split (
    train_dog_cat, label, test_size=0.3, random_state=42)

print("train: ", len(x_train))
print("valid: ", len(x_valid))

train:  140
valid:  60


In [5]:
sess = tf.Session()
model = Model_Vgg19(sess, "model")
model.build_net(150)

Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
learning_rate = 0.001
epochs = 200
batch_size = 1

In [7]:
sess.run(tf.global_variables_initializer())

train_count = len(x_train)
start_time = time.time()

print('Learning started. It takes sometime.') 
for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(train_count / batch_size)
    start_time_epoch = time.time()
    for i in range(total_batch):
        begin_idx = i * batch_size
        end_idx = begin_idx + train_count
        end_idx = min(end_idx, train_count - 1)
        batch_x_train = x_train[begin_idx : end_idx]
        batch_y_train = y_train[begin_idx : end_idx]
        batch_y_train = np.reshape(batch_y_train, [-1,1])
        
        # 리사이즈 이미지 로드
        count = len(batch_x_train)
        data = np.ndarray((count, 150, 150, 3), dtype=np.uint8)

        for i, image_file in enumerate(batch_x_train):
            img = cv2.imread(image_file)
            data[i] = img
        
        # 학습
        c, _ = model.train(data, batch_y_train, learning_rate);
        avg_cost += c / total_batch 
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), " %.2f seconds" % (time.time() - start_time_epoch))

print('Learning Finished!') 
print("--- %.2f seconds ---" %(time.time() - start_time))
        

Learning started. It takes sometime.


ResourceExhaustedError: OOM when allocating tensor with shape[139,64,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: model/conv2d/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model/conv2d/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, model/conv2d/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean/_17 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_774_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, ad